In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%run core_new.py

In [2]:
run_app(query['DSP'], "dsp_kpi")

C:\Users\ssu\Documents\github\alerting\core_new.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = np.vectorize(unit_mm)(df[col])
C:\Users\ssu\Documents\github\alerting\core_new.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = np.vectorize(currency)(df[col])
C:\Users\ssu\Documents\github\alerting\core_new.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [3]:
df = load_data(query['DSP'], "dsp_kpi")

In [4]:
df2 = run_data(df)

C:\Users\ssu\Documents\github\alerting\core_new.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = np.vectorize(unit_mm)(df[col])
C:\Users\ssu\Documents\github\alerting\core_new.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = np.vectorize(currency)(df[col])
C:\Users\ssu\Documents\github\alerting\core_new.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [5]:
df2.head()

,am,email,day,buyerid,offer,offermatchrate,bidrate,timeoutrate,blockrate,winrate,revenue,revcpm,costcpm,margin,offerrate,buyerid,pdbidsofferedtotal,pdoffermatchrate,pdbidrate,pdtimeoutrate,pdblockrate,pdwinrate,pdrevenue,pdrevcpm,pdcostcpm,pdmargin,pdofferrate,buyerid,4davgbidsofferedtotal,4doffermatchrate,4dbidrate,4dtimeoutrate,4dblockrate,4dwinrate,4davgrevenue,4drevcpm,4dcostcpm,4dmargin,4dofferrate,buyerid,7davgbidsofferedtotal,7doffermatchrate,7dbidrate,7dtimeoutrate,7dblockrate,7dwinrate,7davgrevenue,7drevcpm,7dcostcpm,7dmargin,7dofferrate,rev_diff_agst_pd,revenue_change_agst_pd,revenue_change_agst_4davg,revenue_change_agst_7davg,offer_change_agst_pd,offer_change_agst_4davg,offer_change_agst_7davg,offerrate_change_agst_pd,offerrate_change_agst_4davg,offerrate_change_agst_7davg,offermatchrate_change_agst_pd,offermatchrate_change_agst_4davg,offermatchrate_change_agst_7davg,bidrate_change_agst_pd,bidrate_change_agst_4davg,bidrate_change_agst_7davg,blockrate_change_agst_pd,blockrate_change_agst_4davg,blockrate_change_agst_7davg,winrate_change_agst_pd,winrate_change_agst_4davg,winrate_change_agst_7davg,revcpm_change_agst_pd,revcpm_change_agst_4davg,revcpm_change_agst_7davg,costcpm_change_agst_pd,costcpm_change_agst_4davg,costcpm_change_agst_7davg,margin_change_agst_pd,margin_change_agst_4davg,margin_change_agst_7davg,timeoutrate_change_agst_pd,timeoutrate_change_agst_4davg,timeoutrate_change_agst_7davg,weights,weighted_pd_change,weighted_4d_change,weighted_7d_change
advertiser,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
RTB-IPonWeb,Veena,VXavier@pulsepoint.com,2017-03-30,556010,"2,539M",61.0%,25.5%,3.5%,3.4%,1.5%,"&#36;44,290",&#36;4.57,&#36;3.37,26.4%,17.8%,556010,"2,340M",63.0%,30.9%,3.2%,3.2%,1.5%,"&#36;52,860",&#36;4.98,&#36;3.68,26.0%,16.1%,556010,"2,390M",64.0%,28.3%,3.5%,3.4%,1.5%,"&#36;44,563",&#36;4.47,&#36;3.38,24.4%,15.8%,556010,"2,391M",64.0%,30.7%,3.1%,3.3%,1.4%,"&#36;45,843",&#36;4.4,&#36;3.5,20.5%,15.8%,"&#36;-8,570",-16.2%,-0.6%,-3.4%,8.5%,6.2%,6.2%,10.6%,12.7%,12.7%,-3.2%,-4.7%,-4.7%,-17.5%,-9.9%,-16.9%,6.2%,0.0%,3.0%,0.0%,0.0%,7.1%,-8.2%,2.2%,3.9%,-8.4%,-0.3%,-3.7%,1.5%,8.2%,28.8%,9.4%,0.0%,12.9%,2.000000,-32.4%,-1.2%,-6.8%
RTB-SimpliFi,Veena,VXavier@pulsepoint.com,2017-03-30,537085,"5,903M",45.0%,1.1%,1.4%,9.1%,5.5%,"&#36;4,599",&#36;1.31,&#36;1.0,23.4%,41.4%,537085,"5,593M",45.0%,1.1%,1.1%,7.5%,6.1%,"&#36;4,742",&#36;1.3,&#36;0.97,25.3%,38.4%,537085,"5,780M",45.0%,1.0%,1.3%,8.2%,6.4%,"&#36;4,969",&#36;1.33,&#36;1.02,23.2%,38.2%,537085,"5,603M",44.0%,1.1%,1.0%,7.7%,6.4%,"&#36;5,283",&#36;1.4,&#36;1.06,24.2%,36.9%,&#36;-143,-3.0%,-7.4%,-12.9%,5.5%,2.1%,5.3%,7.8%,8.4%,12.2%,0.0%,0.0%,2.3%,0.0%,10.0%,0.0%,21.3%,11.0%,18.2%,-9.8%,-14.1%,-14.1%,0.8%,-1.5%,-6.4%,3.1%,-2.0%,-5.7%,-7.5%,0.9%,-3.3%,27.3%,7.7%,40.0%,1.016686,-3.1%,-7.5%,-13.1%
RTB-Convert-Media,Alexandra,AFrank@pulsepoint.com,2017-03-30,553204,"2,971M",16.0%,22.0%,0.4%,16.5%,0.5%,"&#36;1,291",&#36;0.38,&#36;0.31,19.5%,20.9%,553204,"3,833M",17.0%,23.5%,0.8%,33.0%,0.5%,"&#36;1,968",&#36;0.46,&#36;0.35,22.6%,26.3%,553204,"3,973M",16.0%,21.6%,0.9%,31.9%,0.5%,"&#36;2,137",&#36;0.5,&#36;0.38,24.0%,26.2%,553204,"4,027M",16.0%,21.2%,0.8%,33.5%,0.4%,"&#36;1,730",&#36;0.46,&#36;0.35,24.1%,26.5%,&#36;-677,-34.4%,-39.6%,-25.4%,-22.5%,-25.2%,-26.2%,-20.5%,-20.2%,-21.1%,-5.9%,0.0%,0.0%,-6.4%,1.9%,3.8%,-50.0%,-48.3%,-50.7%,0.0%,0.0%,25.0%,-17.4%,-24.0%,-17.4%,-11.4%,-18.4%,-11.4%,-13.7%,-18.7%,-19.1%,-50.0%,-55.6%,-50.0%,1.078996,-37.1%,-42.7%,-27.4%
RTB-Videology - Video,Ilya,IProvornov@pulsepoint.com,2017-03-30,560244,267M,41.0%,5.2%,1.4%,1.7%,1.0%,"&#36;1,182",&#36;9.02,&#36;6.5,28.0%,1.9%,560244,271M,30.0%,3.2%,1.7%,4.9%,2.0%,"&#36;1,153",&#36;6.67,&#36;4.97,25.5%,1.9%,560244,271M,32.0%,4.0%,1.6%,8.8%,1.7%,"&#36;1,366",&#36;7.5,&#36;5.2,30.7%,1.8%,560244,272M,31.0%,4.2%,1.5%,7.8%,1.7%,"&#36;1,442",&#36;7.45,&#36;5.28,29.1%,1.8%,&#36;29,2.5%,-13.5%,-18.0%,-1.7%,-1.6%,-2.0%,0.0%,5.6%,5.6%,36.7%,28.1%,32.3%,62.5%,30.0%

In [45]:
buyerid = df2['buyerid']
x = buyerid.T.drop_duplicates().T
y = np.array(x['buyerid'])
param = ', '.join([str(i) for i in y])
param

'556010, 537085, 553204, 560244, 559959'

In [ ]:
def load_data(sql, fname):
	with open(sql) as f:
		query = f.read().strip()
	filename = os.path.join(data_directory, "{}_{}.pkl".format(fname, datetime.datetime.now().strftime("%Y%m%d")))
	if os.path.exists(filename) is False:
		df = pd.read_sql(query, create_engine("impala://impala.pulse.prod"))
		df.to_pickle(filename)
	else:
		df = pd.read_pickle(filename)
	return df

In [48]:
import pandas as pd

with open('dsp-pub.sql') as f:
    query = f.read().strip() % (param, param)
    df = pd.read_sql(query, create_engine("impala://impala.pulse.prod"))

select 
        buyerid,
        publisherid,
        ma.accountname as publisher,
        ppdrevenue,
        pdrevenue,
        rev_difference,
        rev_delta,
        offer_difference,
        offer_delta
from
(select
        spend.*,
        rev_difference/ppdrevenue as rev_delta,
        offer_difference,
        offer_delta,
        sum(ppdrevenue) over (partition by spend.buyerid order by ppdrevenue desc) / sum(ppdrevenue) over (partition by spend.buyerid) as cum_pct
from
(select
        buyerid,
        publisherid,
        sum(case when day = date_sub(from_unixtime(iudf.det_unix_timestamp(), 'yyyy-MM-dd'), 2) then revenue else 0 end) as ppdrevenue,
        sum(case when day = date_sub(from_unixtime(iudf.det_unix_timestamp(), 'yyyy-MM-dd'), 1) then revenue else 0 end) as pdrevenue,
        ( sum(case when day = date_sub(from_unixtime(iudf.det_unix_timestamp(), 'yyyy-MM-dd'), 1) then revenue else 0 end) - 
        sum(case when day = date_sub(from_unixtime(iudf.det_unix_times

In [28]:
"My name is %s and i'm %d" % ('john', 12)

"My name is john and i'm 12"

In [ ]:
	day = df['day'][0]
	accts = pd.Series(df.index)
	headers = []
	value = []
	yesterday_cols = ['revenue' , 'offer' , 'offerrate' , 'offermatchrate' , 'bidrate' , 'blockrate' , 'winrate' , 'revcpm'
	                     , 'costcpm' , 'margin' , 'timeoutrate']
	yesterday = df[yesterday_cols]
	yesterday_T = yesterday.T
	delta_col = list(df.loc[:, 'revenue_change_agst_pd':'timeoutrate_change_agst_7davg'])
	delta = df[delta_col]

In [ ]:
	for acct in accts:
		acct_yesterday = yesterday_T[acct]
		pday = slice_data('pd', delta, acct)
		avg4d = slice_data('4davg', delta, acct) 
		avg7d = slice_data('7davg', delta, acct) 
		acct_data = pd.concat([acct_yesterday, pday, avg4d, avg7d], axis = 1, keys=[day, 'change agst prior day', 'change agst 4-day avg', 'change agst 7-day avg'])
		acct_data= acct_data.reset_index()
		data = df_json(acct_data)
		header = data['columns']
		val = data['data']
		headers.append(header)
		value.append(val)

In [ ]:
acct_data

In [ ]:
acct_data['metric'] = acct_data['metric'].apply(lambda x: x.replace('rate', ' rate'))

In [ ]:
acct_data